# Loading libraries

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.nn import Parameter, Sigmoid
from torch.autograd import Variable
import torch.nn as nn

# Network which decides demand and price to bid

In [2]:
class Net(nn.Module):
    """This neural network decides the bid quantity and bid price given the demand, solar output, price predicted by oracle"""
    def __init__(self):
        super(Net, self).__init__()
        self.w1 = Parameter(torch.ones(24) + 0.01 * torch.randn(24))
        self.w2 = Parameter(torch.ones(24) + 0.01 * torch.randn(24))

    def forward(self, oracle_demand, oracle_solar, oracle_price):
        bid_demand = (oracle_demand - oracle_solar) * self.w1
        bid_price = oracle_price * self.w2
        bid_demand.data.clamp(min=0)
        bid_price.data.clamp_(max=7) # As if bid_price is greater than 7 it is beneficial to get electricity from DISCOM.
        return bid_demand, bid_price

# Loading data

In [3]:
demand_train_oracle = pd.read_csv('./Data/Demand_Train_pred.csv', header=None)
demand_train_actual = pd.read_csv('./Data/Demand_Train.csv', header=None)
solar_train_oracle = pd.read_csv('./Data/Solar_Train_pred.csv', header=None)
solar_train_actual = pd.read_csv('./Data/Solar_Train.csv', header=None)
price_train_oracle = pd.read_csv('./Data/Price_Train_pred.csv', header=None)
price_train_actual = pd.read_csv('./Data/Price_Train.csv', header=None)

# Defining cost

In [4]:
def cost(bid_quantities, bid_prices, demands, solar_outputs, prices):
    """Calculates bill given bid quantity, bid price, actual demand, solar output and price."""
    BATTERY_CAPACITY = 25
    BATTERY_CAPACITY_PER_HOUR = 5
    BATTERY_EFFICIENCY = 0.8
    DISCOM_RATE = 7
    battery = Variable(torch.Tensor([0]))
    cost = Variable(torch.Tensor([0]))
    for bid_quantity, bid_price, demand, solar_output, price in zip(bid_quantities.contiguous().view(-1), bid_prices.contiguous().view(-1), demands.contiguous().view(-1), solar_outputs.contiguous().view(-1), prices.contiguous().view(-1)):
        energy_from_sun = torch.min(solar_output, demand)
        sigmoid = Sigmoid()
        bid_won = sigmoid(1000 * (bid_price - price) / bid_price)
        energy_from_market = bid_won * bid_quantity
        energy_from_battery = torch.min(torch.clamp((demand - energy_from_sun - energy_from_market) / BATTERY_EFFICIENCY, min=0),
                                        torch.clamp(battery, max=BATTERY_CAPACITY_PER_HOUR)) * BATTERY_EFFICIENCY
        energy_from_DISCOM = torch.clamp(demand - energy_from_sun - energy_from_market - energy_from_battery, min=0)
        
        residual_energy = torch.clamp(energy_from_market + energy_from_sun - demand, min=0)
        battery = battery + torch.clamp(residual_energy, max=BATTERY_CAPACITY_PER_HOUR) - energy_from_battery / BATTERY_EFFICIENCY
        battery = torch.clamp(battery, min=0, max=BATTERY_CAPACITY)
        
        cost += energy_from_market * bid_price + energy_from_DISCOM * DISCOM_RATE
    return cost

# Training network

In [5]:
data_size = demand_train_actual.values.shape[0]
epochs = 20
batch_size = 1 # Updates weights every day
net = Net()
optimizer = optim.SGD(net.parameters(), lr=10 ** (-8), momentum=0.9)

In [6]:
for epoch in range(epochs):
    start = 0 # starting index
    end = batch_size
    epoch_loss = 0
    while end <= data_size:
        net.zero_grad()
        bid_quantities, bid_prices = net(Variable(torch.Tensor(demand_train_oracle.values[start:end])), 
                                         Variable(torch.Tensor(solar_train_oracle.values[start:end])), 
                                         Variable(torch.Tensor(price_train_oracle.values[start:end])))
        loss = cost(bid_quantities, bid_prices,
                    Variable(torch.Tensor(demand_train_actual.values[start:end])),
                    Variable(torch.Tensor(solar_train_actual.values[start:end])),
                    Variable(torch.Tensor(price_train_actual.values[start:end])))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.data[0]
        start += batch_size
        end += batch_size
    print('Epoch {}, Average bill over 50 days: {}'.format(epoch + 1, epoch_loss * 50 / data_size))

Epoch 1, Average bill over 50 days: 404765.8046875
Epoch 2, Average bill over 50 days: 364841.1834581163
Epoch 3, Average bill over 50 days: 342690.47587076825
Epoch 4, Average bill over 50 days: 330586.8276774089
Epoch 5, Average bill over 50 days: 324875.6608751085
Epoch 6, Average bill over 50 days: 321835.29163953994
Epoch 7, Average bill over 50 days: 320457.3642442491
Epoch 8, Average bill over 50 days: 320186.92351616756
Epoch 9, Average bill over 50 days: 319465.3546685113
Epoch 10, Average bill over 50 days: 319070.78679063584
Epoch 11, Average bill over 50 days: 318964.4822658963
Epoch 12, Average bill over 50 days: 318844.2639160156
Epoch 13, Average bill over 50 days: 318796.81222195097
Epoch 14, Average bill over 50 days: 318721.3420206706
Epoch 15, Average bill over 50 days: 318687.81543646916
Epoch 16, Average bill over 50 days: 318635.1669108073
Epoch 17, Average bill over 50 days: 318567.59283447266
Epoch 18, Average bill over 50 days: 318583.3370700412
Epoch 19, Avera

# Generating solution

## Public leaderboard

In [7]:
demand_pub_oracle = pd.read_csv('./Data/Demand_LB_pred.csv', header=None)
solar_pub_oracle = pd.read_csv('./Data/Solar_LB_pred.csv', header=None)
price_pub_oracle = pd.read_csv('./Data/Price_LB_pred.csv', header=None)

In [8]:
bid_quantities, bid_prices = net(Variable(torch.Tensor(demand_pub_oracle.values)), 
                                 Variable(torch.Tensor(solar_pub_oracle.values)), 
                                 Variable(torch.Tensor(price_pub_oracle.values)))
bid_quantity = np.floor(bid_quantities.data.numpy().flatten())
bid_price = bid_prices.data.numpy().flatten()
submission_pub = pd.DataFrame([bid_price, bid_quantity]).T
submission_pub.to_csv('13_public.csv', header=None, index=None)

# Private leaderboard

In [9]:
demand_prv_oracle = pd.read_csv('./Data/Demand_Test_pred.csv', header=None)
solar_prv_oracle = pd.read_csv('./Data/Solar_Test_pred.csv', header=None)
price_prv_oracle = pd.read_csv('./Data/Price_Test_pred.csv', header=None)

In [10]:
bid_quantities, bid_prices = net(Variable(torch.Tensor(demand_prv_oracle.values)), 
                                 Variable(torch.Tensor(solar_prv_oracle.values)), 
                                 Variable(torch.Tensor(price_prv_oracle.values)))
bid_quantity = np.floor(bid_quantities.data.numpy().flatten())
bid_price = bid_prices.data.numpy().flatten()
submission_prv = pd.DataFrame([bid_price, bid_quantity]).T
submission_prv.to_csv('13_private.csv', header=None, index=None)